# CC Sales Forecasting

**Objective:** Adjust Morgan Stanley's estimates of Safeway monthly sales (sourced from data vendors) using actual CC sales provided by Safeway.

The purpose of this notebook is to illustrate how two companies might use Cape to collaboratively train a linear regression model.

In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns

from cape.api.dataview import DataView
from cape.api.job import VerticalLinearRegressionJob
from cape.cape import Cape

In [2]:
path = Path('data')

# 1. Authenticate with the Cape system

In [3]:
CAPE_TOKEN = os.environ.get("CAPE_TOKEN") or "01EXJB580QC1HJZYPRAY63SMVB,AaDUnYN4PuFtOfEjB1B2gtffK5neUxO2kw"
CAPE_PROJECT = os.environ.get("CAPE_PROJECT") or "01EXJ749YTYF0S8ZJSQ9N1G52V"
CAPE_COORDINATOR = os.environ.get("CAPE_COORDINATOR") or "https://demo.capeprivacy.com"

First, we login to the Cape service via a Python client. This token was generated from the UX, and is unique to the user and organization they're registered with.

In [4]:
c = Cape(endpoint=CAPE_COORDINATOR)
c.login(token=CAPE_TOKEN)

We can look through the projects that I've been assigned as a contributor for. We'll use the `sales-projections` project, which is linked to the Morgan Stanley (`ms`) and Safeway (`sw`) orgs in Cape.

In [5]:
for p in c.list_projects():
    print(p)

project = c.get_project("01EXJ749YTYF0S8ZJSQ9N1G52V")

<Project (id=01EX4XMDF4EM4RDEJ1YZV1E6KA, name=improve-stock-price, label=improve-stock-price)>
<Project (id=01EXFQ4S7J7MVPWHSHDWHP5KDG, name=jason-demo, label=jason-demo)>
<Project (id=01EXFQH15JFGCB209CJDWMCF44, name=stock-projections, label=stock-price-projections)>
<Project (id=01EXJ749YTYF0S8ZJSQ9N1G52V, name=safeway-sales-projections, label=sales-projections)>


## 1. Data Preparation
### a. Morgan Stanley Data

Morgan Stanley's investment arm buys credit card panels from data vendors who work with with financial institutions directly to collect the data. Here are some variables we can expect to find in a credit card panel for one of the portfolio company.
- transaction id
- transaction date
- transaction amount
- merchant description
- mcc
- brand name
- merchant location (address, city, state, state, zip, lat, lon)
- symbol
- card type
- card location (zip, cbsa)
- individual demographics (income, age)

Because the objective is to adjust the dataset at aggregate level (amount by day and state) and not the individual transactions, we created a mock dataset just including the fields: transaction_date, state and ms_transaction_amount.

In [6]:
ms_data = pd.read_csv(path / 'ms' / 'ms_data.csv')
ms_data.head()

,transaction_date,state,ms_transaction_amount
0,2020-01-01,CA,7
1,2020-01-02,CA,7
2,2020-01-03,CA,11
3,2020-01-04,CA,6
4,2020-01-05,CA,8


Because we want to adjust the estimated sales by day/state, we aggregate `ms_transaction_amount` transactions by day/state.

In [7]:
ms_agg = ms_data.groupby(["transaction_date", "state"]).agg({"ms_transaction_amount": "sum"}).rename(columns={"ms_transaction_amount": "ms_total_estimated_sales"})
ms_agg.head()

ms_total_estimated_sales
transaction_date state                          
2020-01-01       CA                           87
                 NY                           94
2020-01-02       CA                           95
                 NY                          100
2020-01-03       CA                           99

Finally, the data owner from Morgan Stanley can create a dataview pointing to the location where this dataset will be stored. In this case, it's in an S3 bucket.

In [8]:
for org in project.organizations:
    print(org)

<Organization (id=01EXJ5WD5WWQKWBKTJXPF18HNK, name=morgan-stanley)>
<Organization (id=01EXJ758K70F0D1QFMQZWW3BNR, name=safeway-tech)>


In [9]:
ms_dataview = DataView(
    name="morgan-stanley-agg-data",
    owner_id=project.organizations[0].id,
    uri='x_data',
    schema=ms_agg.dtypes
)
project.add_dataview(ms_dataview)


<DataView (id=01EXJBFX5VZ48PMNT38VXP06FT, name=morgan-stanley-agg-data, location=None)>

## b. Safeway data

Safeway has access to their actual sales transaction. We can expect the following fields in the dataset:

- transaction id
- transaction date
- location (Safeway only need detail at state level)
- transaction type (credit/debit, cash, etc.)
- transaction amount

For the same reason before, we created a mock dataset just including the fields: transaction_date, state and sw_transaction_amount.

In [10]:
sw_data = pd.read_csv(path / 'sw' / 'sw_data.csv')
sw_data.head()

,transaction_date,state,sw_transaction_amount
0,2020-01-01,CA,15
1,2020-01-02,CA,16
2,2020-01-03,CA,23
3,2020-01-04,CA,14
4,2020-01-05,CA,18


In [11]:
sw_agg = sw_data.groupby(["transaction_date", "state"]).agg({"sw_transaction_amount": "sum"}).rename(columns={"sw_transaction_amount": "sw_total_estimated_sales"})
sw_agg.head()

sw_total_estimated_sales
transaction_date state                          
2020-01-01       CA                          188
                 NY                          204
2020-01-02       CA                          204
                 NY                          216
2020-01-03       CA                          213

In [12]:
for org in project.organizations:
    print(org)

<Organization (id=01EXJ5WD5WWQKWBKTJXPF18HNK, name=morgan-stanley)>
<Organization (id=01EXJ758K70F0D1QFMQZWW3BNR, name=safeway-tech)>


In [14]:
sw_dataview = DataView(
    name="safeway-agg-data",
    owner_id=project.organizations[1].id,
    uri='y_data',
    schema=sw_agg.dtypes
)
project.add_dataview(sw_dataview)


<DataView (id=01EXJBHFHH9BC0HEK6CQ2WTV9F, name=safeway-agg-data, location=None)>

# --- testing SK LEARN results ---
TODO: Remove

In [15]:
X_ms = pd.read_csv(path/'ms/ms_aggregate_data.csv', parse_dates=['transaction_date'])['ms_total_estimated_sales'].to_numpy().reshape(-1, 1)
y_sw = pd.read_csv(path/'sw/sw_aggregate_data.csv', parse_dates=['transaction_date'])['sw_total_estimated_sales'].to_numpy().reshape(-1, 1)

In [16]:

from sklearn.linear_model import  LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

def mse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

def display_coefficients(model, coef_names):
    print("Model Coefficients\n")
    for i, coef in enumerate(coef_names):
        if coef == 'intercept':
            print(f"{coef}: {round(model.intercept_[0], 2)}")
        else:
            print(f"{coef}: {round(model.coef_.tolist()[i][0], 2)}")

In [17]:
# Train linear regression
model = LinearRegression()
model.fit(X_ms, y_sw)

display_coefficients(model, ['ms_total_estimated_sales', 'intercept'])

Model Coefficients

ms_total_estimated_sales: 2.03
intercept: 12.13


In [18]:
pred = model.predict(X_ms)
mse_val = mse(y_sw, pred)
r2 = r2_score(y_sw, pred)

print(f"Train - R-Squared: {r2}")
print(f"Train - RMSE: {mse_val}")

Train - R-Squared: 0.9932724594707781
Train - RMSE: 2.140561949858766


# Morgan Stanley -- Linear Regression Job

Now the data scientist can inspect the project and its available dataviews on their end.

In [27]:
dataviews = project.list_dataviews()
for dv in dataviews:
    print(dv)

<DataView (id=01EXJBFX5VZ48PMNT38VXP06FT, name=morgan-stanley-agg-data, location=None)>
<DataView (id=01EXJBHFHH9BC0HEK6CQ2WTV9F, name=safeway-agg-data, location=None)>


Finally, we can use these dataviews to create a linear regression training job. Since the input variables and output variables are split across different dataviews, we specify this as a `VerticalLinearRegressionJob`. These dataviews are references to datasets that live elsewhere, so we just need to specify which columns in the dataview we want to use for the input and output variables of the model.

In [28]:
job = VerticalLinearRegressionJob(
        x_train_dataview=dataviews[0],
        x_train_data_cols=['ms_total_estimated_sales'],
        y_train_dataview=dataviews[1],
        y_train_data_cols=['sw_total_estimated_sales'],
    )

We then register and create the job with the Cape service. It should now be visible in the Cape UI.

In [29]:

job = project.create_job(job=job)

Lastly, we can submit the job for execution. Under the hood, this sets up a worker Session on the Cape Coordinator, which will be used to orchestrate and compute the encrypted linear regression.

In [37]:
job.submit_job()

<VerticalLinearRegressionJob (id=01EXJBSMXPHEMPNZ00JD0Z5EJV, job_type=LINEAR_REGRESSION)>

We can check the status of this job here, and when it's complete we can fetch the results.


In [ ]:
# TODO